In [ ]:
from sklearn.model_selection import ParameterGrid
import yaml
from subprocess import call

import os

DEST_DIR = 'experiment_results_gridsearch_target_bdd'
os.makedirs(DEST_DIR,exist_ok=True)
    
done_experiments = []
for item in os.listdir(DEST_DIR):
    da_data = { pair.split('_')[0]:pair.split('_')[1:]  for pair  in item.split('__')[0:3] + item.split('__')[14:]     }
    da_coefficients = { pair.split('_')[0]:float(pair.split('_')[1])  for pair  in item.split('__')[5:(5+9)] } 
    da_coefficients['LAMBDA'] = float(da_data['lambda'][0])
    sorted_coeffs = sorted(da_coefficients.items(), key=lambda x:x[0])
    done_experiments.append(str(sorted_coeffs) )



# set coefficients of the variable that are explored by 
# grid search of experiments as one or more point values
da_coefficients_gs = {
 #'res2': [0.0 ,0.5],
 #'res3': [0.0 ,0.5],
 #'res4': [0.0 ,0.5],
 #'res5': [0.0 ,0.5],

 'res2': [0.0 ],
 'res3': [0.0 ],
 'res4': [0.0 ],
 'res5': [0.0 ],

                      
  'p2': [0.0 ,0.5],
 'p3': [0.0 ,0.5],
 'p4': [0.0 ,0.5],
 'p5': [0.0 ,0.5],
 'p6': [0.0, 0.5 ],

'LAMBDA': [1.0 ]
                     }

print('num_experiments: ', len(ParameterGrid(da_coefficients_gs)))


da_yaml_params_dict = {}
with open(r'configs/da_cfg.yaml') as file:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
        da_yaml_params_dict = yaml.load(file, Loader=yaml.FullLoader)


for grid in ParameterGrid(da_coefficients_gs):
        
    # TODO bdd target, cityscapes outofsample     
    reduction_factor = 0
    for k,v in grid.items():
        if v > 0. and v < 1. and k != 'LAMBDA':
            reduction_factor +=1
    for k,v in grid.items():
        if k != 'LAMBDA':
            selected_v = v/max(1,reduction_factor)
            da_yaml_params_dict['DA']['MODEL'][k] = selected_v
            grid[k] = selected_v
    with open(r'configs/da_cfg.yaml', 'w') as file:
        documents = yaml.dump(da_yaml_params_dict, file)

    
    sorted_coeffs = str(sorted(grid.items(), key=lambda x:x[0]))
    if sorted_coeffs not in done_experiments:
        print('Executing job: ',sorted_coeffs)
        call(["python3", "train_tensorboard.py", "--num-gpus", "2"])
    else:
        print('Job already completed: ',sorted_coeffs)